# MCMC3.5: Jackknife Resampling

To estimate the error of observables accurately, it is recommended to use the Jackknife resampling method.

## Binning

Previously, we faced a large errorbar problem at low temperature in this program:

In [1]:
using ResumableFunctions
using SparseArrays
using LinearAlgebra
const Jx = 1 / 3 # oppposite sign to Motome's
const Jy = 1 / 3
const Jz = 1 / 3
function Metropolis(βF::Float64, βFnew::Float64)::Bool
    βF - βFnew > log(rand())
end
function openhoneycomb(Lx::Int64, Ly::Int64)::Tuple
    N = 2Lx * Ly
    nnx = zip(1 : 2 : (N - 1), 2 : 2 : N)
    nny = Iterators.flatten(zip((1 + 2i) : 2Lx : (2Lx * (Ly - 1)  + 1 + 2i), 2i : 2Lx : (2Lx * (Ly - 1)  + 2i)) for i in 1 : (Lx - 1))
    nnz = zip(1 : 2 : (N - 1), Iterators.flatten(((2Lx + 2) : 2 : N, 2 : 2 : 2Lx)))
    plaquette = Iterators.flatten(zip((Lx * (i - 1) + 1) : (Lx * (i - 1) + Lx - 1), (Lx * (i - 1) + 2) : (Lx * (i - 1) + Lx)) for i in 1 : Ly)
    N, nnx, nny, nnz, plaquette
end
@resumable function measurementflux(method::Function, lattice::Function, β::Float64, Lx::Int64, Ly::Int64)::Float64
    N, nnx, nny, nnz, plaquette = lattice(Lx, Ly)
    iter = Iterators.flatten((Iterators.product(J, nn) for (J, nn) in [(Jx, nnx), (Jy, nny), (Jz, nnz)]))
    h = spzeros(Complex{Float64}, N, N)
    for (J, nn) in iter
        h[nn[1], nn[2]] = 0.5im * J
        h[nn[2], nn[1]] = -0.5im * J
    end
    NNz = collect(nnz)
    Nz = length(NNz)
    η = ones(Int64, Nz)
    βF = 0.0
    hdense = Array(h)
    plaq = collect(plaquette)
    Np = length(plaq)
    while true
        for i in 1 : Nz
            j = rand(1 : Nz)
            hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
            hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            ev = eigvals(Hermitian(hdense))
            positiveev = Iterators.drop(ev, N >> 1)
            βFnew = -sum((log(2.0 * cosh(β * ϵ / 2.0)) for ϵ in positiveev))
            if method(βF, βFnew)
                η[j] = -η[j]
                βF = βFnew
            else
                hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
                hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            end
        end
        @yield sum((η[k] * η[l] for (k, l) in plaq)) / Np
    end
end

measurementflux (generic function with 1 method)

One reason is the discreteness of the returned value "flux."

In [2]:
mcstep = Iterators.drop(measurementflux(Metropolis, openhoneycomb, 100.0, 4, 4), 10000)
foreach(println, Iterators.take(mcstep, 10))

0.3333333333333333
0.5
0.16666666666666666
0.3333333333333333
-0.3333333333333333
0.16666666666666666
-0.16666666666666666
0.0
-0.16666666666666666
0.6666666666666666


We have to flatten these quantized values by binning.

In [3]:
using Statistics
Nsample = 10000
Nbin = 100
Nbinsize = Nsample ÷ Nbin
iter = Iterators.partition(Iterators.take(mcstep, Nsample), Nbinsize)
bin = collect(map(mean, iter))

100-element Array{Float64,1}:
 0.25166666666666665
 0.33833333333333326
 0.22500000000000003
 0.2833333333333333 
 0.3083333333333333 
 0.305              
 0.30833333333333335
 0.30833333333333335
 0.3566666666666667 
 0.32166666666666666
 0.35166666666666657
 0.32               
 0.27666666666666667
 ⋮                  
 0.295              
 0.31               
 0.32833333333333337
 0.29               
 0.37666666666666665
 0.3416666666666666 
 0.30666666666666664
 0.26166666666666666
 0.32               
 0.29666666666666663
 0.32833333333333337
 0.3399999999999999 

Now it works!

In [4]:
m = mean(bin)
s = stdm(bin, m) / sqrt(length(bin))
println("$m ± $s")

0.30755 ± 0.0036713663636895426


Nbinsize has to be determined based on the autocorrelation. In order to reduce Nbin to get a more acculate result, we need to implement some global updating algorithm.

## Delete-1 jackknife resampling

Delete-1 jackknife resampling is simply implemented in https://github.com/ararslan/Jackknife.jl. However, the function is limited, so I will newly define functions for the jackknife resampling.

In [5]:
function leaveoneout(before::Function, after::Function, v::AbstractVector)
    ind = eachindex(v)
    map(i -> after(mean(map(before, view(v, filter(!isequal(i), ind))))), ind)
end
meanJ(b::Function, a::Function, v::AbstractVector) = mean(leaveoneout(b, a, v))
stdmJ(b::Function, a::Function, v::AbstractVector, m) = stdm(leaveoneout(b, a, v), m, corrected = false) * sqrt(length(v) - 1)
stdJ(b::Function, a::Function, v::AbstractVector) = stdmJ(b, a, v, meanJ(b, a, v))

stdJ (generic function with 1 method)

The functions are based on Statistics.jl and Jackknife.jl, so please see their reference to know how it works. I again use measurementEf for the demonstration.

In [6]:
@resumable function measurementEf(method::Function, lattice::Function, β::Float64, Lx::Int64, Ly::Int64)::Vector{Float64}
    N, nnx, nny, nnz, plaquette = lattice(Lx, Ly)
    iter = Iterators.flatten((Iterators.product(J, nn) for (J, nn) in [(Jx, nnx), (Jy, nny), (Jz, nnz)]))
    h = spzeros(Complex{Float64}, N, N)
    for (J, nn) in iter
        h[nn[1], nn[2]] = 0.5im * J
        h[nn[2], nn[1]] = -0.5im * J
    end
    NNz = collect(nnz)
    Nz = length(NNz)
    η = ones(Int64, Nz)
    βF = 0.0
    β₂ = β * 0.5
    hdense = Array(h)
    plaq = collect(plaquette)
    Np = length(plaq)
    ev = zeros(Float64, N)
    while true
        for i in 1 : Nz
            j = rand(1 : Nz)
            hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
            hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            evnew = eigvals(Hermitian(hdense))
            βFnew = -sum(@. log(exp(β₂ * evnew[(N >> 1 + 1) : end]) + exp(-β₂ * evnew[(N >> 1 + 1) : end])))
            if method(βF, βFnew)
                η[j] = -η[j]
                βF = βFnew
                ev .= evnew
            else
                hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
                hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            end
        end
        Ef = -sum(@. ev[(N >> 1 + 1) : end] * tanh(β₂ * ev[(N >> 1 + 1) : end] )) * 0.5
        ∂Ef∂β = -sum(@. (ev[(N >> 1 + 1) : end] ^ 2) * (sech(β₂ * ev[(N >> 1 + 1) : end]) ^ 2)) * 0.25
        @yield [Ef, ∂Ef∂β]
    end
end

measurementEf (generic function with 1 method)

It is ok to first assume the bin size to be 1 for this problem.

In [7]:
const β = 10.0
mcstep2 = Iterators.drop(measurementEf(Metropolis, openhoneycomb, β, 4, 4), 10000)
iter2 = Iterators.take(mcstep2, Nsample)
data = collect(iter2)

10000-element Array{Array{Float64,1},1}:
 [-1.68042, -0.0454945]
 [-1.68826, -0.0473381]
 [-1.665, -0.0431975]  
 [-1.69243, -0.0481061]
 [-1.68162, -0.0458104]
 [-1.68441, -0.0469445]
 [-1.68104, -0.045745] 
 [-1.68561, -0.046466] 
 [-1.67757, -0.0450611]
 [-1.69541, -0.0485065]
 [-1.68162, -0.0458104]
 [-1.6796, -0.0459299] 
 [-1.69361, -0.0483618]
 ⋮                     
 [-1.68707, -0.0470462]
 [-1.66452, -0.0429756]
 [-1.68444, -0.0469634]
 [-1.67332, -0.0445042]
 [-1.67932, -0.0457827]
 [-1.68615, -0.0466634]
 [-1.67768, -0.0453661]
 [-1.67805, -0.0454819]
 [-1.68427, -0.0469116]
 [-1.68399, -0.0465644]
 [-1.68104, -0.045745] 
 [-1.69321, -0.048395] 

By setting after = before = identity, meanJ and stdmJ work in the same way as mean and stdm, respectively.

In [8]:
m2 = meanJ(identity, identity, data)
s2 = stdmJ(identity, identity, data, m2)
println("Ef = $(m2[1]) ± $(s2[1]), ∂Ef∂β = $(m2[2]) ± $(s2[2])")

Ef = -1.6833612268417353 ± 7.416156890922378e-5, ∂Ef∂β = -0.0464616986607695 ± 1.3158376480010395e-5


This agrees with the standard estimation method for the error bars.

In [9]:
std(data) / sqrt(length(data))

2-element Array{Float64,1}:
 7.416156890897991e-5
 1.315837648002303e-5

For such mean values (i.e. op = mean), the jackknife resampling is apparently overkill. However, to estimate the error for the values like the specific heat, the jackknife resampling is very effective.

In [10]:
TTCv(v::Vector{Float64}) = [v[1] ^ 2 - v[2], v[1]]
Cv(meanTTCv::Vector{Float64}) = (β ^ 2) * (meanTTCv[1] - meanTTCv[2] ^ 2)
m3 = meanJ(TTCv, Cv, data)
s3 = stdmJ(TTCv, Cv, data, m3)
println("Cv = $m3 ± $s3")

Cv = 4.65166925433098 ± 0.001323276237596492


## Autocorrelation

The simplest way to estimate autocorrelation is by changing the size of binning and estimating its errors by jackknife resampling.

In [11]:
binning = Iterators.partition(data, 2)
bin2 = collect(map(mean, binning))

5000-element Array{Array{Float64,1},1}:
 [-1.68434, -0.0464163]
 [-1.67871, -0.0456518]
 [-1.68302, -0.0463774]
 [-1.68332, -0.0461055]
 [-1.68649, -0.0467838]
 [-1.68061, -0.0458702]
 [-1.68946, -0.0477477]
 [-1.68109, -0.0460505]
 [-1.68057, -0.0458423]
 [-1.68067, -0.0459857]
 [-1.68146, -0.0465996]
 [-1.68538, -0.0471572]
 [-1.68998, -0.0475709]
 ⋮                     
 [-1.678, -0.0457511]  
 [-1.68925, -0.0473677]
 [-1.68163, -0.0458801]
 [-1.68356, -0.046485] 
 [-1.67763, -0.0456946]
 [-1.68292, -0.0462062]
 [-1.67579, -0.0450109]
 [-1.67888, -0.0457338]
 [-1.68273, -0.046223] 
 [-1.67787, -0.045424] 
 [-1.68413, -0.046738] 
 [-1.68712, -0.04707]  

In [12]:
m4 = meanJ(TTCv, Cv, bin2)
s4 = stdmJ(TTCv, Cv, bin2, m4)
println("Cv = $m4 ± $s4")

Cv = 4.64899491259832 ± 0.0013320820507892847


The fact that the binsize does not affect the expectation value (or the errorbar) too much means that the autocorrelation length is about 1-2. Note that at low temperature the binsize strongly affects the expectation value, which means that the binsize must be taken to be large enough. Here we recalculate the autocorrelation length by another method.